# Past Match Data Mining Script

- Run this Notebook step wise only, the steps are numbered on top of each cell
- before running the script update the CityUrls csv file where fill  'Y'  in the cities whose data is to be scraped and  'N'  to skip that city
- After Updating the file running STEP 4 is a must And Save file as <b>.csv</b> only
- Scraped data will be saved in the ScrapedData Folder
- The Function in STEP 6 will inform if the Mining of that city's data was successful or not if not run the script again after updating the url file where mark 'N' where mining was sucessful  

In [1]:
## STEP 1 ##

import time
import os
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
## STEP 2 ##

result_path = "../ScrapedData" #Location where Scraped Data will be stored
driver_path = "../Driver/chromedriver.exe" #Location of WebDriver

In [3]:
## STEP 3 ##

#Importing Web Driver
#!! Important Note !!#
# The Chrome Driver Version Should Match the current Version of Google chrome currently installed
# Check Current Version by Options -> Help -> About Chrome
#Current Webdriver Version = 81
#Download Updated version if Necessary from --> https://chromedriver.chromium.org/downloads

try:
    driver = webdriver.Chrome(driver_path)
    driver.close()
    print('Webdriver Loaded Successfully')
except Exception as e:
    print(e)

Webdriver Loaded Successfully


In [4]:
## STEP 4 ##

#Importing URL file
try:
    urldf = pd.read_csv('CityUrls.csv')
    print('URL file Loaded')
except Exception as e:
    print(e)
toscrape = urldf[urldf.SCRAPE == 'Y'].reset_index(drop = True)

URL file Loaded


In [12]:
## STEP 5 ##

def ScrapeCityData(city, url):
    try:
        driver = webdriver.Chrome(driver_path)
        driver.get(url)
        time.sleep(10)
        print('\nScraping Data of :',city)
    except Exception as e:
        print(e)
        return 0
        
    ##### Load More Loop #####
    LoadMoreid = "loadMoreMatches"
    val = " "
    print('Loading Data...')
    while(val!='display: none;'):
        try:
            #waitForLoad(LOAD_MORE_BUTTON_XPATH)
            #WebDriverWait(driver, PTIME)
            loadMoreButton = driver.find_element_by_id(LoadMoreid)
            val = loadMoreButton.get_attribute('style')
            print(val)
            driver.execute_script("arguments[0].click();", loadMoreButton)
            print('Button Clicked')
            time.sleep(5)
        except Exception as e:
            print(e)
            break
    print("Loading data Complete")


    res = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit()
    soup = BeautifulSoup(res)
    box = soup.find_all('div',{'class':'row matchesDiv'})
    
    # Extract Data From the HTML Source page
    AllMatches = []
    for matches in box:
        for match in matches:
            try:
                MatchDetails = match.find('div',{'class':'pmd-card pmd-card-media-inline pmd-card-default pmd-z-depth'})
                
                MatchDetails = None
            if(MatchDetails is None):
                continue
            else:
                try:
                    matchName = MatchDetails.contents[1].contents[1].contents[0].contents[0]  
                except:
                    matchName = 'None'
                try:
                    Address = MatchDetails.contents[3].contents[3].contents[1].contents[0]
                    print(Address)
                except:
                    Address = 'None'
                try:
                    teamA = MatchDetails.contents[5].contents[1].contents[1].contents[1].contents[1].contents[0]
                except:
                    teamA = 'None'
                try:    
                    scoreA = MatchDetails.contents[5].contents[1].contents[1].contents[3].contents[1].contents[0].contents[0]
                except:
                    scoreA = 'None'
                try:
                    teamB = MatchDetails.contents[5].contents[3].contents[1].contents[1].contents[1].contents[0]
                except:
                    teamB = 'None'
                try:
                    scoreB = MatchDetails.contents[5].contents[3].contents[1].contents[3].contents[1].contents[0].contents[0]
                except:
                    scoreB = 'None'
                try:
                    winner = MatchDetails.contents[7].contents[1].contents[0].contents[0]
                except:
                    winner = 'None'
                try:
                    wonBy = MatchDetails.contents[7].contents[1].contents[2].contents[0]
                except:
                    wonBy = 'None'
                try:
                    matchLink = "https://cricheroes.in" + match.contents[1].get('href')
                except:
                    matchLink = 'None'
                oneMatch = [matchName, Address, teamA, scoreA, teamB, scoreB, winner, wonBy, matchLink]
                AllMatches.append(oneMatch)
    
    #Saving File
    try:
        mdf = pd.DataFrame(AllMatches, columns = ['MatchName', 'Address', 'TeamA', 'ScoreA', 'TeamB', 'ScoreB', 'Winner','WonBy', 'MatchLink'])
        mdf.to_csv(os.path.join(result_path,city+'.csv'), index = False)
        print('Data of',city,' Extracted and File Saved Successfully\n\n')
        return 1
    except Exception as e:
        print(e)
        return 0

In [13]:
## STEP 6 ##

for i in range(toscrape.shape[0]):
    city = toscrape.CITY.iloc[i].replace(' ','_')
    url = toscrape.URL.iloc[i]
    #print('Scraping Data of', city)
    status = ScrapeCityData(city, url)
    if(status == 0):
        print('Data Scraping Unsucessful of City :', city)
    elif(status == 1):
        print('Data Scraping Sucessful of City :', city)
    else:
        print('Error Occoured:',status)


Scraping Data of : Ahmedabad
Loading Data...

Button Clicked

Button Clicked

Button Clicked

Button Clicked

Button Clicked

Button Clicked


KeyboardInterrupt: 